<a href="https://colab.research.google.com/github/vintagedeek/mnist/blob/master/kaggle_mnist_feed_forward_one_hidden_no_cnn_2020_07_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
import numpy as np

import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [42]:
train_url = "https://raw.githubusercontent.com/wehrley/Kaggle-Digit-Recognizer/master/train.csv"
df_train = pd.read_csv(train_url)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [43]:
test_url = 'https://raw.githubusercontent.com/wehrley/Kaggle-Digit-Recognizer/master/test.csv'
df_test = pd.read_csv(test_url)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [44]:
train_set = df_train[0:29400]
train = torch.tensor(np.array(train_set.iloc[:, 1:]), dtype=torch.torch.float32)
train_labels = torch.tensor(np.array(train_set.iloc[:, 0]), dtype=torch.int64)
val_set = df_train[29400:]
val = torch.tensor(np.array(val_set.iloc[:, 1:]), dtype=torch.float32)
val_labels = torch.tensor(np.array(val_set.iloc[:, 0]), dtype=torch.int64)

In [45]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [46]:
device

'cuda'

In [47]:
train_labels.shape

torch.Size([29400])

In [65]:
batch_size = 50
epochs = 70
learning_rate = 0.1
lmbda = 0
hidden_neurons_size = 100
in_features = 784 # image is shape (28, 28)

In [66]:
network = nn.Sequential(
    nn.Linear(in_features=in_features, out_features=hidden_neurons_size),
    nn.Sigmoid(),
    nn.Linear(in_features=hidden_neurons_size, out_features=10),
    nn.Sigmoid())

In [67]:
train_labels.shape

torch.Size([29400])

In [68]:
train.shape

torch.Size([29400, 784])

In [69]:
network = network.to(device)
optimizer = optim.SGD(network.parameters(), lr=learning_rate, weight_decay=lmbda)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
for j in range(epochs):
    num_correct = 0
    val_number_correct = 0
    mini_batches = [train[k:k + batch_size] for k in range(0, train_labels.shape[0], batch_size)]
    mini_labels = [train_labels[k:k + batch_size] for k in range(0, train_labels.shape[0], batch_size)]
    for b in range(0, len(mini_batches)):
        images = mini_batches[b].to(device)
        #print(images.shape)
        labels = mini_labels[b].to(device)
        #print(labels.shape)
        #print(labels.dtype)
        #print(images.dtype)
        preds = network(images)
        loss = F.cross_entropy(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        num_correct += preds.argmax(dim=1).eq(labels).sum().item()
      
    validation_set = val.to(device)
    validation_labels = val_labels.to(device) 
    validation_preds = network(validation_set)
    val_number_correct += validation_preds.argmax(dim=1).eq(validation_labels).sum().item()

    print('Completed epoch: ', j)
    print('Train accuracy: ', num_correct/len(train))
    print('Validation Accuracy: ', val_number_correct/len(validation_set))
      
    #scheduler.step()

Completed epoch:  0
Train accuracy:  0.8193197278911565
Validation Accuracy:  0.8787301587301587
Completed epoch:  1
Train accuracy:  0.8866666666666667
Validation Accuracy:  0.8894444444444445
Completed epoch:  2
Train accuracy:  0.8960544217687075
Validation Accuracy:  0.9004761904761904
Completed epoch:  3
Train accuracy:  0.9030612244897959
Validation Accuracy:  0.9046825396825396
Completed epoch:  4
Train accuracy:  0.9071428571428571
Validation Accuracy:  0.9046031746031746
Completed epoch:  5
Train accuracy:  0.9091156462585034
Validation Accuracy:  0.9030952380952381
Completed epoch:  6
Train accuracy:  0.9099659863945578
Validation Accuracy:  0.9107936507936508
Completed epoch:  7
Train accuracy:  0.9134013605442177
Validation Accuracy:  0.9135714285714286
Completed epoch:  8
Train accuracy:  0.9144897959183673
Validation Accuracy:  0.9175396825396825
Completed epoch:  9
Train accuracy:  0.9163945578231293
Validation Accuracy:  0.9156349206349207
Completed epoch:  10
Train acc

In [70]:
# Train accuracy
num_correct/len(train)

0.9464625850340136

In [71]:
len(train)

29400

In [72]:
# val accuracy
val = val.to(device)
val_labels = val_labels.to(device)
val_preds = network(val)
val_num_correct = val_preds.argmax(dim=1).eq(val_labels).sum().item()
val_num_correct/len(val_preds)

0.939047619047619

## Train/Validation accuracy
- 93%/90.9%: bs=50, ep=70, lr=.00001, lmda=0, hidden=100, 1-hidden sigmoid Adam
- 24% for both when using SGD instead of Adam and all other params above same

- 70%/70%: bs=50, ep=70, lr=.00001, lmda=1, hidden=100, 1-hidden sigmoid Adam

- 11%/11%: bs=50, ep=70, lr=.00001, lmda=10.0, hidden=100, 1-hidden sigmoid Adam

- 90.8/90.1: bs=50, ep=70, lr=.00001, lmda=0.1, hidden=100, 1-hidden sig adam

- 92.9%/90.8%: bs=50, ep=70, lr=.00001, lmda=0.01, hidden=100, 1-hidden sigmoid Adam

- 93.1/90.8%: bs=50, ep=70, lr=.00001, lmda=0.001, hidden=100, 1-hidden sig Adam

- 91.1/89.7%: bs=50, ep=40, lr=.00001, lmda=0, hidden=100, 1-hidden sigmoid Adam

- 89.1%/87.2%: bs=50, ep=70, lr=.00001, lmda=0, hidden=30, 1-hidden sigmoid Adam

- lr=3 got us to 9.4% train/val with 50, 70, 0, 100, 1-hidden sig, Adam

- lr=3 got us to 20% train/val with 50, 70, 0, 100, 1-hidden sig, SGD, not adam

- lr=1 got us to 70% train/val with 50, 70, 0, 100, 1-hidden sig, SGD, not adam




- **94.6/94.0: bs=50, ep=70, lr=.1, lmda=0, hidden=100, 1-hidden sigmoid SGD, no batchnorm1D.** (for this one, if you remove last sigmoid activation function, training and val accuracy drop to about 87%).
 - Adding batch norm to first layer resulted in training/val acc about 87%.
 - When add batch norm to both layers, train/val acc 84/82

- **96/92.9: bs=50, ep=70, lr=.01, lmda=0, hidden=100, 1-hidden sigmoid SGD. No batchnorm1D.**


### This gets 11 percent accuracy

    network = nn.Sequential(
    nn.Linear(in_features=in_features, out_features=hidden_neurons_size),
    nn.ReLU(),
    nn.Linear(in_features=hidden_neurons_size, out_features=10))#,
    #nn.Sigmoid())

### Train 56% and val 47% with

    network = nn.Sequential(
    nn.Linear(in_features=in_features, out_features=hidden_neurons_size),
    nn.ReLU(),
    nn.Linear(in_features=hidden_neurons_size, out_features=10),
    nn.Sigmoid())

### The successful runs used this architecture:
    network = nn.Sequential(
    nn.Linear(in_features=in_features, out_features=hidden_neurons_size),
    nn.Sigmoid(),
    nn.Linear(in_features=hidden_neurons_size, out_features=10),
    nn.Sigmoid())


### Network below with negative log likelihood cost F.nll_loss(preds, targets) gets training/val accuracy around 83%, but was still increasing when training stopped at 70 epochs.Tried with 120 epochs and similar results, about 84%.
    network = nn.Sequential(
    nn.Linear(in_features=in_features, out_features=hidden_neurons_size),
    nn.Sigmoid(),
    nn.Linear(in_features=hidden_neurons_size, out_features=10),
    nn.Softmax())


- nn.Tanh had low 70 percent accuracies with same other parameters as when Sigmoid neurons had 90% accuracy. Why?



- 91.2/89.5: bs=50, ep=40, lr=.001, lmda=0, hidden=100, 1-hidden sigmoid SGD



- fell to 60s with lambda=1 and then to 11% with lambda=10
